# Proyecto E-Commerce

Liga con documentos a utilizar (Descargar): https://drive.google.com/drive/folders/1QW-li7nyZwfKUMTxHL2uiNFBtFqxLesN?usp=sharing

**Funciones para sacar matrices de distancia y tiempo**

In [ ]:
import pandas as pd

import time
import requests
import json

from typing import List

BING_MAPS_API = "As9KGSS7Uuz4ycaDG5UxNG-zo1J7xN6LZSTZx1SFjHzBzA_p1hlgmwQteBLpw4Iu"

API_URL_MATRICES = f"https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrixAsync?key={BING_MAPS_API}"
API_URL_LOCATIONS = f"https://dev.virtualearth.net/REST/v1/Locations"

POST_HEADERS = {"content-length": "2500",
                "content-type": "application/json"}


def get_distance_matrix_payload(origin_locations: pd.DataFrame,
                                destin_locations: pd.DataFrame,
                                time_unit:str = "second",
                                distance_unit:str = "km",
                                start_time:str = "2023-08-01T10:00:00-06:00") -> dict:

    """"
    Función que genera el formato necesario para utilizar la API de Bing Maps para el cálculo de las matrices de distancia.
    SÓLO ACEPTA 50 DISTINTOS ORÍGENES Y DESTINOS PARA GENERAR UNA MATRIZ DE DISTANCIAS (25 x 25)

    origin_locations (pd.DataFrame): Dataframe que incluye columnas 'latitude' 'longitude' de los nodos de donde partirá un agente.
    destin_locations (pd.DataFrame): Dataframe que incluye columnas 'latitude' 'longitude' de los nodos de donde se llegará de los origin_location.
    time_unit (str): string que indica las unidades de tiempo para la matriz de tiempo entre nodos.
    distance_unit (str): string que indica las unidades de distancia para la matriz de distancia entre nodos.
    start_time (str): string que indica el tiempo del cual se simulará las distancias y tiempos entre los nodos.

        RETURNS

    payload (dict): diccionario con el formulario necesario para usar método POST de la API de Bing
    """


    origin_point_locations = [{"latitude": lat, "longitude": lon} for lat, lon in zip(origin_locations["latitude"], origin_locations["longitude"])]

    destin_point_locations = [{"latitude": lat, "longitude": lon} for lat, lon in zip(destin_locations["latitude"], destin_locations["longitude"])]

    payload = dict(origins = origin_point_locations,
                   destinations = destin_point_locations,
                   travelMode = "driving",
                   timeUnit = time_unit,
                   distanceUnit = distance_unit,
                   startTime = start_time)

    return payload

def cost_matrices_100_nodes(payloads: List[dict]):
    """
    Función para generar matriz de distancias y de tiempos según una lista de los formatos necesarios.
    SOLO ACEPTA PARA GENERAR UNA MATRIZ DE 100 x 100 utilizando 4 payloads de 25 x 25

    payloads (list[dict]): formatos de entrada para la utilización de la API de Bing. Se debió haber usado la función de get_distance_matrix_payload() para generar 4 payloads distinitos

        RETURNS

    pd.DataFrame(distance_data): Dataframe que incluye la matriz de distancia entre 100 puntos

    pd.DataFrame(time_data): Dataframe que incluye la matriz de tiempos entre 100 puntos

    response_log: Lista que incluye los 'logs' de las respuestas dadas por la API
    """

    response_log = []

    distance_data = {i: [] for i in range(100)}
    time_data = {i: [] for i in range(100)}

    for n, payload in enumerate(payloads, start=1):
        #print("Realizando request de matriz de distancia")
        response = requests.post(API_URL_MATRICES, json=payload, headers=POST_HEADERS).json()

        #print("Cargando matriz de distancias")

        tries = 0
        while True:
            try:
                matrix_url = f"https://routematrixpremium.blob.core.windows.net/finalresults/{response['resourceSets'][0]['resources'][0]['requestId']}"
                distance_matrix_json = json.loads(requests.get(matrix_url).text.encode().decode("utf-8-sig"))
                break
            except:
                #print("Esperando a cargar matriz...")
                tries += 1

                if tries == 10:
                    raise BaseException(f"Timeout Error: {tries*3} seconds passed without response")
                time.sleep(3)

                continue

        #print("Guardando resultado en lista 'response_log'")
        response_log.append(distance_matrix_json)

        #print("Guardando resultados de distancia y tiempos en formato para pd.DataFrame...")

        for distance in distance_matrix_json["results"]:
            origin_index = int(distance["originIndex"])
            destin_index = int(distance["destinationIndex"])

            d_ij = int(distance["travelDistance"])
            t_ij = int(distance["travelDuration"])

            match n:
                case 1:
                    distance_data[destin_index].append(d_ij)
                    time_data[destin_index].append(t_ij)

                case 2:
                    distance_data[destin_index + 50].append(d_ij)
                    time_data[destin_index + 50].append(t_ij)

                case 3:
                    distance_data[destin_index].append(d_ij)
                    time_data[destin_index].append(t_ij)

                case 4:
                    distance_data[destin_index + 50].append(d_ij)
                    time_data[destin_index + 50].append(t_ij)

    return pd.DataFrame(distance_data), pd.DataFrame(time_data), response_log

def VRP_cost_matrix(locations: pd.DataFrame):
    distance_payload_1_1 = get_distance_matrix_payload(origin_locations=locations.iloc[:50],
                                                       destin_locations=locations.iloc[:50])

    distance_payload_1_2 = get_distance_matrix_payload(origin_locations=locations.iloc[:50],
                                                       destin_locations=locations.iloc[50:])

    distance_payload_2_1 = get_distance_matrix_payload(origin_locations=locations.iloc[50:],
                                                       destin_locations=locations.iloc[:50])

    distance_payload_2_2 = get_distance_matrix_payload(origin_locations=locations.iloc[50:],
                                                       destin_locations=locations.iloc[50:])

    payloads = [distance_payload_1_1,
                distance_payload_1_2,
                distance_payload_2_1,
                distance_payload_2_2]

    return cost_matrices_100_nodes(payloads)

In [ ]:
lat_lon = pd.read_excel("/content/lat_lon.xlsx")
lat_lon = lat_lon[["latitude", "longitude"]]

## **Simulación**

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy import stats
import matplotlib.pyplot as plt
import random

In [ ]:
df = pd.read_csv("frecuencia_unidades.csv")
sum_unidades_por_producto = pd.read_csv("frecuencia_productos.csv")
info_productos = pd.read_csv("info_productos.csv")
info_productos.columns.to_list()

['Producto', ' Volumen_mtro3']

In [ ]:
# FUNCIONES

# Función para determinar el número de unidades basado en la probabilidad aleatoria
def determinar_unidades(probabilidad_aleatoria, probabilidad_df):
    for index, row in probabilidad_df.iterrows():
        if row['Limite inferior'] <= probabilidad_aleatoria <= row['Limite Superior']:
            return row['Unidades']

def determinar_volumen(df_info_productos, simulacion):
  Simulacion["Vector_Probabilidades_Aleatorias"] = Simulacion["Demanda"].apply(lambda demanda: np.random.rand(demanda))
  accumulated_frequencies = np.array(sum_unidades_por_producto["Frecuencia_Relativa_Acumulada"])
  n = len(accumulated_frequencies)

  nombre_producto = np.array(info_productos["Producto"].astype(int))
  labels_dict = {i:label for i, label in enumerate(nombre_producto)}

  Simulacion["Productos_Indices_Comprados"] = Simulacion.apply(lambda row: np.searchsorted(accumulated_frequencies, row["Vector_Probabilidades_Aleatorias"]), axis=1)
  Simulacion["Productos_Comprados"] = Simulacion["Productos_Indices_Comprados"].apply(lambda index: np.vectorize(labels_dict.get)(index) if len(index) > 0 else [])

  # Volumen
  volumen_producto = np.array(info_productos[" Volumen_mtro3"])
  diccionario_productos = dict(zip(info_productos["Producto"].astype(int), info_productos[" Volumen_mtro3"]))

  Simulacion["Volumen"] = Simulacion["Productos_Comprados"].apply(lambda index: np.vectorize(diccionario_productos.get)(index) if len(index) > 0 else [])
  Simulacion["Volumen_Total"] = Simulacion["Volumen"].apply(lambda vector: sum(vector))

  resumen_simulacion_df = Simulacion[['Cliente', 'Demanda', 'Productos_Comprados', 'Volumen_Total']]

  return resumen_simulacion_df


# FUNCIÓN VRP CON HEURÍSTICA DE GREEDY
def greedy_vrp_min_distancia(matriz_distancia, matriz_tiempo, demanda, volumen, capacidad, tiempo_max, n_camiones):
    n_clientes = len(matriz_distancia)
    no_visitados = set(range(1, n_clientes)) # nodos que no han sido visitados
    rutas = []

    camiones_utilizados = 0

    for i in range(n_camiones):
        nodo_actual = 0  # Empieza del depósito (nodo 0)
        capacidad_restante = capacidad
        tiempo_actual = 0

        ruta = [nodo_actual]  # Debe empezar y terminar en el depósito (nodo 0)

        while no_visitados and tiempo_actual < tiempo_max:
            # generar nodos_factibles que cheque los nodos no visitados y los mete a los nodos factibles
            # si el volumen del nodo es menor a la capacidad restante y la demanda debe ser mayor de 0
            # para que no visite los nodos que no tengan demanda
            nodos_factibles = [
                nodo for nodo in no_visitados if volumen[nodo] <= capacidad_restante and demanda[nodo] > 0
            ]
            # si no tiene nodos factibles ya se sale del ciclo
            if not nodos_factibles:
                break

            # elegir el nodo factible que tenga la mínima distancia
            nodo_sig = min(nodos_factibles, key=lambda nodo: matriz_distancia[nodo_actual][nodo])

            # checar condiciones del siguiente nodo
            if tiempo_actual + matriz_tiempo[nodo_actual][nodo_sig] + matriz_tiempo[nodo_sig][0] <= tiempo_max:
                capacidad_restante -= volumen[nodo_sig]
                tiempo_actual += matriz_tiempo[nodo_actual][nodo_sig]
                nodo_actual = nodo_sig
                ruta.append(nodo_sig)
                no_visitados.remove(nodo_sig)
            else:
                break

        ruta.append(0)  # Regresa al depósito (nodo 0) cuando ya no se puede añadir más
        if len(ruta) == 2:
          break
        rutas.append(ruta)
        camiones_utilizados += 1

    return rutas, camiones_utilizados

# FUNCIÓN VRP CON HEURÍSTICA DE GREEDY ALEATORIA
def greedy_aleatorio_vrp_min_distancia(matriz_distancia, matriz_tiempo, demanda, volumen, capacidad, tiempo_max, n_camiones):
    n_clientes = len(matriz_distancia)
    no_visitados = set(range(1, n_clientes)) # nodos que no han sido visitados
    rutas = []

    camiones_utilizados = 0

    for i in range(n_camiones):
        nodo_actual = 0  # Empieza del depósito (nodo 0)
        capacidad_restante = capacidad
        tiempo_actual = 0

        ruta = [nodo_actual]  # Debe empezar y terminar en el depósito (nodo 0)

        while no_visitados and tiempo_actual < tiempo_max:
            # generar nodos_factibles que cheque los nodos no visitados y los mete a los nodos factibles
            # si el volumen del nodo es menor a la capacidad restante y la demanda debe ser mayor de 0
            # para que no visite los nodos que no tengan demanda
            nodos_factibles = [
                nodo for nodo in no_visitados if volumen[nodo] <= capacidad_restante and demanda[nodo] > 0
            ]
            if len(nodos_factibles) >= 3:
              nodo_sig = sorted(range(len(nodos_factibles)), key=lambda nodo: matriz_tiempo[nodo_actual][nodo])
              nodo_sig = random.choice(nodos_factibles[:3])
            #Una vez que los nodos factibles sean menores a 3 elige de manera aleatoria dentro de los restantes mas proximos
            elif len(nodos_factibles) < 3 and len(nodos_factibles) > 0:
              nodo_sig = sorted(range(len(nodos_factibles)), key=lambda nodo: matriz_tiempo[nodo_actual][nodo])
              nodo_sig = random.choice(nodos_factibles[:len(nodos_factibles)])
            if not nodos_factibles:
                break
            # si no tiene nodos factibles ya se sale del ciclo
            if not nodos_factibles:
                break

            # checar condiciones del siguiente nodo
            if tiempo_actual + matriz_tiempo[nodo_actual][nodo_sig] + matriz_tiempo[nodo_sig][0] <= tiempo_max:
                capacidad_restante -= volumen[nodo_sig]
                tiempo_actual += matriz_tiempo[nodo_actual][nodo_sig]
                nodo_actual = nodo_sig
                ruta.append(nodo_sig)
                no_visitados.remove(nodo_sig)
            else:
                break

        ruta.append(0)  # Regresa al depósito (nodo 0) cuando ya no se puede añadir más
        if len(ruta) == 2:
          break
        rutas.append(ruta)
        camiones_utilizados += 1

    return rutas, camiones_utilizados


# FUNCIÓN PARA CALCULAR LOS COSTOS DE LAS RUTAS
def costo_ruta(ruta, matriz_distancia, matriz_tiempo, volumen):
    distancia_total = 0
    tiempo_total = 0
    volumen_total = 0

    for i in range(len(ruta) - 1):
        nodo_actual = ruta[i]
        nodo_sig = ruta[i + 1]
        distancia_total += matriz_distancia[nodo_actual][nodo_sig]
        tiempo_total += matriz_tiempo[nodo_actual][nodo_sig]
        volumen_total += volumen[nodo_sig]

    return distancia_total, tiempo_total, volumen_total

In [ ]:
n_sim = 1000 # NÚMERO DE SIMULACIONES

# Agarramos 100 direcciones aleatorias para crear matriz de distancia y tiempo
# direcciones_escogidas = lat_lon.sample(100)

capacidad = 11.5
tiempo_max = 19800
n_camiones = 10

# ESTADÍSTICAS DE VRP GREEDY
todos_distancia_total_greedy = []
todos_tiempo_total_greedy = []
total_porcentaje_volumen_no_utilizado_greedy = []
total_porcentaje_tiempo_no_utilizado_greedy = []
total_camiones_utilizados_greedy = []
todos_tiempo_ultima_ruta_greedy = []

# ESTADÍSTICAS DE VRP GREEDY ALEATORIO
todos_distancia_total_aleatorio = []
todos_tiempo_total_aleatorio = []
total_porcentaje_volumen_no_utilizado_aleatorio = []
total_porcentaje_tiempo_no_utilizado_aleatorio = []
total_camiones_utilizados_aleatorio = []
todos_tiempo_ultima_ruta_aleatorio = []

for i in range(n_sim):
  # CADA 100 ITERACIONES CAMBIAN LOS CLIENTES

  if i % 100 == 0:

    print("SE CAMBIARON LOS CLIENTES")
    tec = pd.DataFrame(lat_lon.iloc[0]).T
    direcciones_escogidas = pd.concat([tec, lat_lon.sample(99)])
    matriz_distancia, matriz_tiempo, _ = VRP_cost_matrix(direcciones_escogidas)


  # DETERMINAMOS NÚMERO DE UNIDADES POR CLIENTE
  n=100
  clientes = [i for i in range (n)]
  nodos = [0] + clientes

  #DEMANDA
  demanda=np.array([determinar_unidades(np.random.rand(),df) for n in clientes])
  demanda[0] = 0

  data_clientes = {'Cliente': clientes, 'Demanda': demanda}
  Simulacion = pd.DataFrame(data_clientes)

  Simulacion["Demanda"] = Simulacion["Demanda"].astype(int)

  resumen_simulacion = determinar_volumen(info_productos, Simulacion)

  # Variables para usar en el greedy
  demanda = np.array(resumen_simulacion["Demanda"])
  volumen = np.array(resumen_simulacion["Volumen_Total"])

  # USAMOS VRP CON GREEDY
  rutas, camiones_utilizados = greedy_vrp_min_distancia(matriz_distancia, matriz_tiempo, demanda, volumen, capacidad, tiempo_max, n_camiones)

  distancia_total_greedy = []
  tiempo_total_greedy = []
  volumen_total_greedy = []
  ultima_rutas= []

  for i, ruta in enumerate(rutas):
      distancia, tiempo, volume = costo_ruta(ruta, matriz_distancia, matriz_tiempo, volumen)
      #print(f"Ruta {i + 1}: Nodos {ruta}, Distancia: {distancia} km, Tiempo: {round(tiempo/3600,2)} hrs, Volumen: {volume} m^3") # QUITAR COMENTARIO SI QUEREMOS VER DETALLE DE CADA RUTA
      distancia_total_greedy.append(distancia)
      tiempo_total_greedy.append(tiempo)
      volumen_total_greedy.append(volume)
      if i == camiones_utilizados-1:
        ultima_rutas.append(tiempo)


  #print("----------------------------------------------------") # QUITAR COMETARIO SI QUIEREMOS VER DETALLE DE CADA RUTA

  todos_distancia_total_greedy.append(sum(distancia_total_greedy))
  todos_tiempo_total_greedy.append(sum(tiempo_total_greedy))

  porcentaje_volumen_no_utilizado_greedy = round(((((capacidad * camiones_utilizados) - sum(volumen_total_greedy)) / (capacidad * camiones_utilizados))*100),2)
  total_porcentaje_volumen_no_utilizado_greedy.append(round(porcentaje_volumen_no_utilizado_greedy, 2))

  porcentaje_tiempo_no_utilizado_greedy = round(((((tiempo_max * camiones_utilizados) - sum(tiempo_total_greedy)) / (tiempo_max * camiones_utilizados))*100), 2)
  total_porcentaje_tiempo_no_utilizado_greedy.append(round(porcentaje_tiempo_no_utilizado_greedy, 2))

  total_camiones_utilizados_greedy.append(camiones_utilizados)

  todos_tiempo_ultima_ruta_greedy.append(sum(ultima_rutas))



  # USAMOS VRP CON GREEDY ALEATORIO
  rutas, camiones_utilizados = greedy_aleatorio_vrp_min_distancia(matriz_distancia, matriz_tiempo, demanda, volumen, capacidad, tiempo_max, n_camiones)

  distancia_total_aleatorio = []
  tiempo_total_aleatorio = []
  volumen_total_aleatorio = []

  for i, ruta in enumerate(rutas):
      distancia, tiempo, volume = costo_ruta(ruta, matriz_distancia, matriz_tiempo, volumen)
      #print(f"Ruta {i + 1}: Nodos {ruta}, Distancia: {distancia} km, Tiempo: {round(tiempo/3600,2)} hrs, Volumen: {volume} m^3") # QUITAR COMENTARIO SI QUEREMOS VER DETALLE DE CADA RUTA
      distancia_total_aleatorio.append(distancia)
      tiempo_total_aleatorio.append(tiempo)
      volumen_total_aleatorio.append(volume)

  #print("----------------------------------------------------") # QUITAR COMETARIO SI QUIEREMOS VER DETALLE DE CADA RUTA

  todos_distancia_total_aleatorio.append(sum(distancia_total_aleatorio))
  todos_tiempo_total_aleatorio.append(sum(tiempo_total_aleatorio))

  porcentaje_volumen_no_utilizado = round(((((capacidad * camiones_utilizados) - sum(volumen_total_aleatorio)) / (capacidad * camiones_utilizados))*100),2)
  total_porcentaje_volumen_no_utilizado_aleatorio.append(round(porcentaje_volumen_no_utilizado, 2))

  porcentaje_tiempo_no_utilizado = round(((((tiempo_max * camiones_utilizados) - sum(tiempo_total_aleatorio)) / (tiempo_max * camiones_utilizados))*100), 2)
  total_porcentaje_tiempo_no_utilizado_aleatorio.append(round(porcentaje_tiempo_no_utilizado, 2))

  total_camiones_utilizados_aleatorio.append(camiones_utilizados)

SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES


In [ ]:
# GREEDY RESULTADOS
promedio_distancia_greedy = np.mean(todos_distancia_total_greedy)
promedio_tiempo_greedy = np.mean(todos_tiempo_total_greedy)

promedio_porcentaje_volumen_no_utilizado_greedy = np.mean(total_porcentaje_volumen_no_utilizado_greedy)
promedio_porcentaje_tiempo_no_utilizado_greedy = np.mean(total_porcentaje_tiempo_no_utilizado_greedy)

promedio_camiones_utilizados_greedy = np.mean(total_camiones_utilizados_greedy)

promedio_ultima_rutas = np.mean(todos_tiempo_ultima_ruta_greedy)

print("RESULTADOS DE VRP CON GREEDY")
print(f"Promedio de distancia total recorrida por cada iteración: {promedio_distancia_greedy} km")
print(f"Promedio de tiempo total recorrido por cada iteración: {round(promedio_tiempo_greedy/3600, 2)} hrs")

print(f"Promedio de % de volumen no utilizado por cada iteración: {round(promedio_porcentaje_volumen_no_utilizado_greedy, 2)} %")
print(f"Promedio de % de tiempo no utilizado por cada iteración: {round(promedio_porcentaje_tiempo_no_utilizado_greedy, 2)} %")

print(f"Promedio de rutas utilizados por cada iteración: {promedio_camiones_utilizados_greedy} rutas")

print(f"Promedio de tiempo de ultimas rutas: {round(promedio_ultima_rutas/3600,2)} hrs")

print("----------------------------------------------------")


# GREEDY ALEATORIO
promedio_distancia_aleatorio = np.mean(todos_distancia_total_aleatorio)
promedio_tiempo_aleatorio = np.mean(todos_tiempo_total_aleatorio)

promedio_porcentaje_volumen_no_utilizado_aleatorio = np.mean(total_porcentaje_volumen_no_utilizado_aleatorio)
promedio_porcentaje_tiempo_no_utilizado_aleatorio = np.mean(total_porcentaje_tiempo_no_utilizado_aleatorio)

promedio_camiones_utilizados_aleatorio = np.mean(total_camiones_utilizados_aleatorio)

print("RESULTADOS DE VRP CON GREEDY ALEATORIO")
print(f"Promedio de distancia total recorrida por cada iteración: {promedio_distancia_aleatorio} km")
print(f"Promedio de tiempo total recorrido por cada iteración: {round(promedio_tiempo_aleatorio/3600, 2)} hrs")

print(f"Promedio de % de volumen no utilizado por cada iteración: {round(promedio_porcentaje_volumen_no_utilizado_aleatorio, 2)} %")
print(f"Promedio de % de tiempo no utilizado por cada iteración: {round(promedio_porcentaje_tiempo_no_utilizado_aleatorio, 2)} %")

print(f"Promedio de rutas utilizados por cada iteración: {promedio_camiones_utilizados_aleatorio} rutas")


RESULTADOS DE VRP CON GREEDY
Promedio de distancia total recorrida por cada iteración: 426.424 km
Promedio de tiempo total recorrido por cada iteración: 16.8 hrs
Promedio de % de volumen no utilizado por cada iteración: 65.06 %
Promedio de % de tiempo no utilizado por cada iteración: 11.83 %
Promedio de rutas utilizados por cada iteración: 3.506 rutas
Promedio de tiempo de ultimas rutas: 3.38 hrs
----------------------------------------------------
RESULTADOS DE VRP CON GREEDY ALEATORIO
Promedio de distancia total recorrida por cada iteración: 1756.479 km
Promedio de tiempo total recorrido por cada iteración: 47.51 hrs
Promedio de % de volumen no utilizado por cada iteración: 87.48 %
Promedio de % de tiempo no utilizado por cada iteración: 10.14 %
Promedio de rutas utilizados por cada iteración: 9.622 rutas


# **RAMPAS DEFINIDAS**

In [ ]:
# FUNCIÓN VRP CON HEURÍSTICA DE GREEDY
def greedy_vrp_min_distancia_rampas(matriz_distancia, matriz_tiempo, demanda, volumen, capacidad, tiempo_max, n_camiones, rampas):
    n_clientes = len(matriz_distancia)
    no_visitados = set(range(1, n_clientes)) # nodos que no han sido visitados
    rutas = []

    camiones_utilizados = 0

    for i in range(n_camiones):
        nodo_actual = 0  # Empieza del depósito (nodo 0)
        capacidad_restante = capacidad

        if i < rampas:
          tiempo_actual = 0
        else:
          tiempo_actual = 3600

        ruta = [nodo_actual]  # Debe empezar y terminar en el depósito (nodo 0)

        while no_visitados and tiempo_actual < tiempo_max:
            # generar nodos_factibles que cheque los nodos no visitados y los mete a los nodos factibles
            # si el volumen del nodo es menor a la capacidad restante y la demanda debe ser mayor de 0
            # para que no visite los nodos que no tengan demanda
            nodos_factibles = [
                nodo for nodo in no_visitados if volumen[nodo] <= capacidad_restante and demanda[nodo] > 0
            ]
            # si no tiene nodos factibles ya se sale del ciclo
            if not nodos_factibles:
                break

            # elegir el nodo factible que tenga la mínima distancia
            nodo_sig = min(nodos_factibles, key=lambda nodo: matriz_distancia[nodo_actual][nodo])

            # checar condiciones del siguiente nodo
            if tiempo_actual + matriz_tiempo[nodo_actual][nodo_sig] + matriz_tiempo[nodo_sig][0] <= tiempo_max:
                capacidad_restante -= volumen[nodo_sig]
                tiempo_actual += matriz_tiempo[nodo_actual][nodo_sig]
                nodo_actual = nodo_sig
                ruta.append(nodo_sig)
                no_visitados.remove(nodo_sig)
            else:
                break

        ruta.append(0)  # Regresa al depósito (nodo 0) cuando ya no se puede añadir más
        if len(ruta) == 2:
          break
        rutas.append(ruta)
        camiones_utilizados += 1

    return rutas, camiones_utilizados

In [ ]:
n_sim = 1000 # NÚMERO DE SIMULACIONES

# Agarramos 100 direcciones aleatorias para crear matriz de distancia y tiempo
# direcciones_escogidas = lat_lon.sample(100)

capacidad = 11.5
tiempo_max = 19800
n_camiones = 10

rampas = 2

# ESTADÍSTICAS DE VRP GREEDY
todos_distancia_total_greedy = []
todos_tiempo_total_greedy = []
total_porcentaje_volumen_no_utilizado_greedy = []
total_porcentaje_tiempo_no_utilizado_greedy = []
total_camiones_utilizados_greedy = []
todos_tiempo_ultima_ruta_greedy = []

# ESTADÍSTICAS DE VRP GREEDY ALEATORIO
todos_distancia_total_aleatorio = []
todos_tiempo_total_aleatorio = []
total_porcentaje_volumen_no_utilizado_aleatorio = []
total_porcentaje_tiempo_no_utilizado_aleatorio = []
total_camiones_utilizados_aleatorio = []
todos_tiempo_ultima_ruta_aleatorio = []

for i in range(n_sim):
  # CADA 100 ITERACIONES CAMBIAN LOS CLIENTES

  if i % 100 == 0:
    print("SE CAMBIARON LOS CLIENTES")
    tec = pd.DataFrame(lat_lon.iloc[0]).T
    direcciones_escogidas = pd.concat([tec, lat_lon.sample(99)])
    matriz_distancia, matriz_tiempo, _ = VRP_cost_matrix(direcciones_escogidas)


  # DETERMINAMOS NÚMERO DE UNIDADES POR CLIENTE
  n=100
  clientes = [i for i in range (n)]
  nodos = [0] + clientes

  #DEMANDA
  demanda=np.array([determinar_unidades(np.random.rand(),df) for n in clientes])
  demanda[0] = 0

  data_clientes = {'Cliente': clientes, 'Demanda': demanda}
  Simulacion = pd.DataFrame(data_clientes)

  Simulacion["Demanda"] = Simulacion["Demanda"].astype(int)

  resumen_simulacion = determinar_volumen(info_productos, Simulacion)

  # Variables para usar en el greedy
  demanda = np.array(resumen_simulacion["Demanda"])
  volumen = np.array(resumen_simulacion["Volumen_Total"])

  # USAMOS VRP CON GREEDY
  rutas, camiones_utilizados = greedy_vrp_min_distancia_rampas(matriz_distancia, matriz_tiempo, demanda, volumen, capacidad, tiempo_max, n_camiones, rampas)

  distancia_total_greedy = []
  tiempo_total_greedy = []
  volumen_total_greedy = []
  ultima_rutas= []

  for i, ruta in enumerate(rutas):
      distancia, tiempo, volume = costo_ruta(ruta, matriz_distancia, matriz_tiempo, volumen)
      #print(f"Ruta {i + 1}: Nodos {ruta}, Distancia: {distancia} km, Tiempo: {round(tiempo/3600,2)} hrs, Volumen: {volume} m^3") # QUITAR COMENTARIO SI QUEREMOS VER DETALLE DE CADA RUTA
      distancia_total_greedy.append(distancia)
      tiempo_total_greedy.append(tiempo)
      volumen_total_greedy.append(volume)
      if i == camiones_utilizados-1:
        ultima_rutas.append(tiempo)


  #print("----------------------------------------------------") # QUITAR COMETARIO SI QUIEREMOS VER DETALLE DE CADA RUTA

  todos_distancia_total_greedy.append(sum(distancia_total_greedy))
  todos_tiempo_total_greedy.append(sum(tiempo_total_greedy))

  porcentaje_volumen_no_utilizado_greedy = round(((((capacidad * camiones_utilizados) - sum(volumen_total_greedy)) / (capacidad * camiones_utilizados))*100),2)
  total_porcentaje_volumen_no_utilizado_greedy.append(round(porcentaje_volumen_no_utilizado_greedy, 2))

  porcentaje_tiempo_no_utilizado_greedy = round(((((tiempo_max * camiones_utilizados) - sum(tiempo_total_greedy)) / (tiempo_max * camiones_utilizados))*100), 2)
  total_porcentaje_tiempo_no_utilizado_greedy.append(round(porcentaje_tiempo_no_utilizado_greedy, 2))

  total_camiones_utilizados_greedy.append(camiones_utilizados)

  todos_tiempo_ultima_ruta_greedy.append(sum(ultima_rutas))


SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES
SE CAMBIARON LOS CLIENTES


In [ ]:
# GREEDY RESULTADOS
promedio_distancia_greedy = np.mean(todos_distancia_total_greedy)
promedio_tiempo_greedy = np.mean(todos_tiempo_total_greedy)

promedio_porcentaje_volumen_no_utilizado_greedy = np.mean(total_porcentaje_volumen_no_utilizado_greedy)
promedio_porcentaje_tiempo_no_utilizado_greedy = np.mean(total_porcentaje_tiempo_no_utilizado_greedy)

promedio_camiones_utilizados_greedy = np.mean(total_camiones_utilizados_greedy)

promedio_ultima_rutas = np.mean(todos_tiempo_ultima_ruta_greedy)

print("RESULTADOS DE VRP CON GREEDY CON RAMPAS")
print(f"Promedio de distancia total recorrida por cada iteración: {promedio_distancia_greedy} km")
print(f"Promedio de tiempo total recorrido por cada iteración: {round(promedio_tiempo_greedy/3600, 2)} hrs")

print(f"Promedio de % de volumen no utilizado por cada iteración: {round(promedio_porcentaje_volumen_no_utilizado_greedy, 2)} %")
print(f"Promedio de % de tiempo no utilizado por cada iteración: {round((1-((promedio_tiempo_greedy+rampas*3600)/(camiones_utilizados*tiempo_max)))*100, 2)} %")

print(f"Promedio de rutas utilizados por cada iteración: {promedio_camiones_utilizados_greedy} rutas")

print(f"Promedio de tiempo de ultimas rutas: {round((promedio_ultima_rutas+rampas*3600)/3600,2)} hrs")

RESULTADOS DE VRP CON GREEDY CON RAMPAS
Promedio de distancia total recorrida por cada iteración: 434.994 km
Promedio de tiempo total recorrido por cada iteración: 16.97 hrs
Promedio de % de volumen no utilizado por cada iteración: 70.06 %
Promedio de % de tiempo no utilizado por cada iteración: 13.76 %
Promedio de rutas utilizados por cada iteración: 4.0 rutas
Promedio de tiempo de ultimas rutas: 3.97 hrs
